# Library

In [1]:
import os
import torch 
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import pandas as pd

# Preprocessed data

In [2]:
transform = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
        transforms.RandomAffine(degrees=30, translate=(0.2, 0.2), scale=(0.8, 1.2)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Load Data

In [3]:
data_dir = "/kaggle/input/fruits-dataset"
image_datasets = {
    "train": datasets.ImageFolder(os.path.join(data_dir, "train-20230708T062807Z-001/train"), transform["train"]),
    "val": datasets.ImageFolder(os.path.join(data_dir, "validation-20230708T062810Z-001/validation"), transform["val"]),
    "test": datasets.ImageFolder(os.path.join(data_dir, "test-20230708T062801Z-001/test"), transform["test"])
}
print("Class-to-Index Mapping (train):", image_datasets["train"].class_to_idx)


Class-to-Index Mapping (train): {'freshapples': 0, 'freshbanana': 1, 'freshoranges': 2, 'rottenapples': 3, 'rottenbanana': 4, 'rottenoranges': 5}


In [4]:

from torch.utils.data import random_split
#create number of test set
val_size = len(image_datasets["val"]) - 500  
test_size = 500
image_datasets["val"], image_datasets["test"] = random_split(
    image_datasets["val"], [val_size, test_size]
)

data_loaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=(x == "train"))
    for x in ["train", "val", "test"]
}


In [5]:
#check the size of datasets
for phase in ["train", "val", "test"]:
    print(f"{phase} dataset size: {len(image_datasets[phase])}")

train dataset size: 2778
val dataset size: 1942
test dataset size: 500


# PreTrained Model

In [6]:
# use Resnet18 to train but be not used pretrained model
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(image_datasets["train"].classes))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


# Loss function and optimizer

In [9]:
from collections import Counter
train_class_counts = Counter([label for _, label in image_datasets["train"].samples])
total_train_samples = sum(train_class_counts.values())
weights = {cls: total_train_samples / count for cls, count in train_class_counts.items()}
class_weights = torch.tensor([weights[i] for i in range(len(weights))], dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
print("Class weights:", class_weights)

Class weights: tensor([6.0391, 7.2344, 5.1160, 5.8239, 6.0921, 6.0655], device='cuda:0')


# Define Train Model 

In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_model_wts = None
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print("-" * 10)

        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in data_loaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            
            if phase == "val" and epoch_acc > best_val_acc:
                best_val_acc = epoch_acc
                best_model_wts = model.state_dict().copy()

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == "train":
                scheduler.step()

    return model


In [11]:
model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

Epoch 1/25
----------
train Loss: 1.5814 Acc: 0.3711
val Loss: 1.2181 Acc: 0.5505
Epoch 2/25
----------
train Loss: 1.1734 Acc: 0.5554
val Loss: 0.9250 Acc: 0.6905
Epoch 3/25
----------
train Loss: 1.0331 Acc: 0.6138
val Loss: 0.8521 Acc: 0.6864
Epoch 4/25
----------
train Loss: 0.9321 Acc: 0.6494
val Loss: 1.0123 Acc: 0.6014
Epoch 5/25
----------
train Loss: 0.8516 Acc: 0.6969
val Loss: 0.9060 Acc: 0.7425
Epoch 6/25
----------
train Loss: 0.8554 Acc: 0.6782
val Loss: 1.0836 Acc: 0.6256
Epoch 7/25
----------
train Loss: 0.7810 Acc: 0.7196
val Loss: 0.7598 Acc: 0.7302
Epoch 8/25
----------
train Loss: 0.6905 Acc: 0.7617
val Loss: 0.4605 Acc: 0.8522
Epoch 9/25
----------
train Loss: 0.6710 Acc: 0.7703
val Loss: 0.4770 Acc: 0.8419
Epoch 10/25
----------
train Loss: 0.6800 Acc: 0.7545
val Loss: 0.4928 Acc: 0.8399
Epoch 11/25
----------
train Loss: 0.6217 Acc: 0.7851
val Loss: 0.4857 Acc: 0.8378
Epoch 12/25
----------
train Loss: 0.6224 Acc: 0.7797
val Loss: 0.4857 Acc: 0.8368
Epoch 13/25
-

# Define evaluate model 


In [12]:
def evaluate_model_with_report(model, loader):
    from sklearn.metrics import classification_report
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())


    print(classification_report(all_labels, all_preds, target_names=image_datasets["train"].classes))
    return sum(1 for p, l in zip(all_preds, all_labels) if p == l) / len(all_labels)


In [13]:
test_accuracy = evaluate_model_with_report(model, data_loaders["test"])
print(f"Final Test Accuracy: {test_accuracy * 100:.2f}%")

               precision    recall  f1-score   support

  freshapples       0.92      0.88      0.90        81
  freshbanana       0.83      0.99      0.90        82
 freshoranges       0.92      0.75      0.82        72
 rottenapples       0.72      0.86      0.78        84
 rottenbanana       0.93      0.99      0.96        85
rottenoranges       0.92      0.73      0.81        96

     accuracy                           0.86       500
    macro avg       0.87      0.86      0.86       500
 weighted avg       0.87      0.86      0.86       500

Final Test Accuracy: 86.40%
